### Descripcion del notebook

El objetivo de este notebook es generar un dataframe que contenga información de todos los productos en todos los días. 

El dataframe que se usa como fuente es el que se genera en el notebook `C2`

__IMPORTANTE:__ Debido a que el proceso es muy lento, y el tiempo es reducido, el dataframe de este notebook no se ha usado en los modelos.  

In [ ]:
from tqdm import tqdm
import pandas as pd
import numpy as np

Cargo el dataframe que contiene la curva de stock para cada dia y producto

In [ ]:
df_1 = pd.read_csv("train_grouped2.csv",index_col=[0,1])

Obtengo las ids unicas de los productos que están en todo el dataframe

In [ ]:
lista_ids = df_1.index.get_level_values(1).unique().to_list()

In [ ]:
dia_final = df_1.index.get_level_values(0).unique().max()

Creo un dataframe de la siguiente estructura.

* Las filas representan los días del dataframe (64 días). 
* Las columnas representan los productos del dataframe (8722 productos).

Cada celda puede tener 2 valores, `True` o `False`. 
* Si hay un `True` significa que para ese producto en ese dia, existe un valor en el daframe
* Si hay un `False`, por el contrario, significa que ese producto en ese día no tiene valor en el dataframe

Para obtener los valores, recorro el dataframe

In [ ]:
ids_products = lista_ids

incompletos = {}
for j in tqdm(ids_products):
    existe = []
    for i in range(0,dia_final+1):
        existe.append(df_1.index.isin([(i, j)]).any())
    incompletos[j] = existe

curvas = pd.DataFrame(incompletos)
# pd.DataFrame(curvas).to_csv("info_curva_informada.csv",header=True)

Selecciono los registros que necesita que se les cree algún registro

In [ ]:
dia_con_huecos = []
for i in tqdm(curvas.columns):
    x = curvas[i] == False
    if (x.unique().shape[0] == 2) or ((x.unique().shape[0] == 1) & (x.unique()[0] == True)):
        dia_con_huecos.append(i)

En `vacios_completos` están las ids de los productos que necesita algún registro para tener la curva de stocks completa (de 0 a 63)

Creo los registros que faltan con  `NaN`

In [ ]:
for id_producto in tqdm(dia_con_huecos):
    producto_curvas = curvas[id_producto]  
    for index,value in enumerate(producto_curvas.to_list()):
        # Selecciono solo los False
        if not value:
            df_1.loc[(index,id_producto),:] = None

Finalmente, reconstruimos la curva, rellenando los datos a nulo. 

Este proceso sigue esta lógica:

- Los campos `price`,`block_id`,`family_id`,`subfamily_id` contienen información que no cambia en el tiempo. Se rellenan los nulos con el único valor que existe para ese producto en toda la linea temporal con este esquema:
```py
subconjunto.loc[:,campo].fillna(subconjunto[campo].dropna().unique()[0],inplace=True)
```

- Los campos `color_id`,`size_id`,`stock` se rellenan a partir de la información que exista de ellos. Si no existen en el día 0, se inicializan con valor 0.0. Se sigue la estrategia de reemplazo `foward fill`. Así, si un producto aparece por primera vez en el día 7, por ejemplo, se asume que no existen en la tienda hasta ese día, por lo que decir que su stock es 0 es correcto. A partir de ahí, si un producto no realiza una venta un día, se puede asumir que tiene el mismo stock que el día anterior.   

- El campo `sales` se rellena siempre a 0.0 si no existe en un día (si es NaN).  


In [ ]:
for id_producto in tqdm(dia_con_huecos):
    # Selecciono todos los días de un producto. 
    prueba_train_expandido = df_1.loc[pd.IndexSlice[:,id_producto], :]

    prueba_train_expandido.loc[:,'price'].fillna(prueba_train_expandido['price'].dropna().unique()[0],inplace=True)
    prueba_train_expandido.loc[:,'block_id'].fillna(prueba_train_expandido['block_id'].dropna().unique()[0],inplace=True)
    prueba_train_expandido.loc[:,'family_id'].fillna(prueba_train_expandido['family_id'].dropna().unique()[0],inplace=True)
    prueba_train_expandido.loc[:,'subfamily_id'].fillna(prueba_train_expandido['subfamily_id'].dropna().unique()[0],inplace=True)

    if prueba_train_expandido.loc[0,'color_id'].any() == False:
        prueba_train_expandido.loc[0,'color_id'] = 0.0

    if prueba_train_expandido.loc[0,'size_id'].any() == False:
        prueba_train_expandido.loc[0,'size_id']= 0.0

    if prueba_train_expandido.loc[0,'stock'].any() == False:
        prueba_train_expandido.loc[0,'stock'] = 0.0


    prueba_train_expandido['color_id'].fillna(method='ffill',inplace=True)
    prueba_train_expandido['size_id'].fillna(method='ffill',inplace=True)
    prueba_train_expandido['sales'].fillna(0.0,inplace=True)
    prueba_train_expandido['stock'].fillna(method='ffill',inplace=True)
   
  

Finalmente, ordenamos el dataframe 

In [ ]:
df_1.sort_index(inplace=True)

Guardo el dataframe en un csv

In [ ]:
df_1.to_csv("full_csv_stocks.csv",header=True)